In [7]:
from azure.storage.blob import BlockBlobService
import pandas as pd
from pandas import DataFrame
import numpy as np
import json
import os
import requests
from itertools import islice

import csv

from datetime import datetime, timedelta

from matplotlib import pyplot as plt
%matplotlib inline

In [8]:
def formatter(s):
    '''
    Trasforma la data ""%Y-%m-%d %H:%M:%S*" in "%Y-%m-%d %H:%M:%S.%f"
    '''
    try:
        n = len(s.split('.')[1])
    except:
        s = s+'.'
        n = 0
    for i in range(4-n):
        s = s+'0'
    return s

def finder_pos(dataframe, when):
    '''
    Se un timestamp è assente, inserisce un record vuoto.
    '''
    try:
        stamp = dataframe.loc[when]['datetime_copy']
    except:
        stamp = ''
    try:
        lat = dataframe.loc[when]['latitude']
    except:
        lat = ''
    try:
        lon = dataframe.loc[when]['longitude']
    except:
        lon = ''
    try:
        vel = dataframe.loc[when]['speed']
    except:
        vel = ''
    try:
        road = dataframe.loc[when]['road']
    except:
        road = ''
    try:
        limit = dataframe.loc[when]['speed_limit']
    except:
        limit = ''
    return stamp,lat,lon,vel,road,limit

In [9]:
# SET PROXY TO REACH BLOB
proxy_cred = pd.read_csv('proxy.cred.txt',
                         sep=';',
                         encoding='UTF8',
                         header=None,
                         index_col=0)

proxy = proxy_cred.loc['proxy'][1]
os.environ['http_proxy'] = proxy
os.environ['https_proxy'] = proxy
requests.get("http://google.com")

<Response [200]>

In [10]:
#Azure blob configuration
credentials = pd.read_csv('blobazure_iot_dopo_di_agosto.cred',
                          sep=';',
                          encoding='UTF8',
                          header=None,
                          index_col=0)
account = credentials.loc['account'][1]
key = credentials.loc['key'][1]

account = account
key = key
generali_blob = BlockBlobService(account_name=account,
                                 sas_token=key)

In [12]:
'''
GENERA IL NEXT MARKER DELLA PAGINA CHE SI VUOLE VISITARE (page_to_visit)
'''
page_to_visit = 2
num_pages = 0
next_marker = None

path = 'infomobility/TSP_VOUCHER'
container = 'crash'

for i in range(page_to_visit):  
    num_pages +=1
    print(num_pages)
    generator = generali_blob.list_blobs(container,
                                         path,
                                         marker=next_marker)
    next_marker = generator.next_marker
    print(next_marker)
    if len(next_marker)<10:
        break

1
2!176!MDAwMDg3IWluZm9tb2JpbGl0eS9UU1BfVk9VQ0hFUi9JTkZPTU9CSUxJVFkyNTMyMDEvMjAxOC85LzI3LzIwMThfMDlfMjdfMTVfMzVfNDhfNTQ5NjQzNTIuanNvbiEwMDAwMjghOTk5OS0xMi0zMVQyMzo1OTo1OS45OTk5OTk5WiE-
2
2!176!MDAwMDg3IWluZm9tb2JpbGl0eS9UU1BfVk9VQ0hFUi9JTkZPTU9CSUxJVFkyNjQ1MTkvMjAxOC8xMC80LzIwMThfMTBfMDRfMDlfNTJfMzNfNTU1NjYwMTcuanNvbiEwMDAwMjghOTk5OS0xMi0zMVQyMzo1OTo1OS45OTk5OTk5WiE-


#### PAGINATION

In [ ]:
'''
Genera due csv per crash: un _POS sui dati del gps, uno _ACC sui dati dell'accelerometro
'''

path = 'infomobility/TSP_VOUCHER'
container = 'crash'

crash = pd.DataFrame()
counter = 0
blob2parse = 50000


while True:
    generator = generali_blob.list_blobs(container,
                                         path,
                                         marker=next_marker)

    df_pos = pd.DataFrame()
    totali = len(generator.items)
    print(totali)

    for blob in generator.items:
#------# se errore, salta il crash        
        try:
            counter = counter + 1
            if counter > blob2parse:
                break

            if counter%10==0:
                print('Parsati almeno %i blob' %(counter))

            name = blob.name
            
            generali_blob.get_blob_to_path(container,
                                           name,
                                           name.split('/')[1])
            
   #--------# LETTURA DEI DATI, INFO GENERICHE E PREPARAZIONE LISTE
            data=json.load(open(name.split('/')[1]))

            # info da header e placeholder per id crash
            deviceId = data['header']['deviceId']
            tripId =  data['header']['tripId']
            voucherId = data['header']['voucherId']
            batchId = data['header']['batchId']
            crashId = 'PLACEHOLDER_'+str(counter)

            # definizione crash time in formato microsec
            crash_time = data['header']['crashTimestamp']
            crash_time = crash_time.split('T')[0]+' '+crash_time.split('T')[1]
            crash_time = datetime.strptime(formatter(crash_time), '%Y-%m-%d %H:%M:%S.%f')

            # conterrà le info di accelerometro e gps da trasformare in dataframe
            acc = []
            pos = []

   #--------# POSITION
            for sample in data['crashPosition']:
                date_time = ' '.join(sample['time'].split('T'))
                date_time = formatter(date_time)  # formatta la data per to_datetime
                lat = sample['lat']
                lon = sample['lon']
                speed = sample['speed']
                road = sample['enrichment']['roadType']
                limit = sample['enrichment']['speedLimit']
                pos.append([date_time,
                            lat,
                            lon,
                            speed,
                            road,
                            limit
                           ])
            pos = DataFrame(pos, columns = ['datetime',
                                            'latitude',
                                            'longitude',
                                            'speed',
                                            'road',
                                            'speed_limit'
                                           ])

            pos['datetime'] = pd.to_datetime(pos['datetime'],
                                             format='%Y-%m-%d %H:%M:%S.%f')
            pos['datetime_copy'] = pos['datetime']
            pos = pos.set_index('datetime')
        
            # inserisce record vuoti per timestamp mancanti
            frames = []
            for i in range(-20,21):
                t = crash_time + timedelta(seconds=i)          
                stamp, latit, longit, veloc, road, lim = finder_pos(pos,t)
                frames.append([
                    stamp,
                    i,
                    latit,
                    longit,
                    veloc,
                    road,
                    lim,
                    deviceId,
                    tripId,
                    voucherId,
                    batchId,
                    crashId
                    ])
            frame = DataFrame(frames, columns = [
                                            'datetime',
                                            'seconds_to_crash',
                                            'latitude',
                                            'longitude',
                                            'speed',
                                            'road',
                                            'limit',
                                            'deviceId',
                                            'tripId',
                                            'voucherId',
                                            'batchId',
                                            'crashId'
                                            ])

   #--------# ACCELERATION
            for sample in data['crashAcceleration']:
                date_time = ' '.join(sample['time'].split('T'))
                date_time = formatter(date_time)  # formatta la data per to_datetime
                x = sample['acc']['x']
                y = sample['acc']['y']
                z = sample['acc']['z']
                acc.append([date_time,
                            x,
                            y,
                            z,
                            deviceId,
                            tripId,
                            voucherId,
                            batchId,
                            crashId])
            acc = DataFrame(acc, columns = ['datetime',
                                            'accX',
                                            'accY',
                                            'accZ',
                                            'deviceId',
                                            'tripId',
                                            'voucherId',
                                            'batchId',
                                            'crashId'])  
            acc['datetime'] = pd.to_datetime(acc['datetime'],
                                             format='%Y-%m-%d %H:%M:%S.%f')
            idx = acc[acc['datetime']==crash_time].index.values[0]
            for i in range(idx+1, len(acc)):
                t2 = acc.iloc[i]['datetime']
                t1 = acc.iloc[i-1]['datetime']
                delta = (t2-t1).microseconds/1000

            file_name = '_'.join([name.split('/')[6].strip('.json'), deviceId, tripId])
            frame.to_csv('C:\\Users\leuci\Desktop\Blob2csv\POS\\'+file_name+'_POS.csv',index=False,sep=';')
            acc.to_csv('C:\\Users\leuci\Desktop\Blob2csv\ACC\\'+file_name+'_ACC.csv',index=False,sep=';')
            
#------# SE ERRORE, SALTA...     
        except Exception as e:
            counter += 1
            print('error {} on {}'.format(e, name))
            pass
#---# FINITA LA PAGINA. PASSA ALLA PAGINA SUCCESSIVA
    next_marker = generator.next_marker
    print('Switch to page', next_marker)
    
    if counter > blob2parse or next_marker==None:  # Se ha parsato i blob richiesti o le pagine sono terminate
        break

print('Finito. Parsati {} blob.'.format(counter-1))
filepage.close()

print('Pos mancanti:',len(df_pos[df_pos['latitude']==''])/len(df_pos))

5000
Parsati almeno 10 blob
Parsati almeno 20 blob
Parsati almeno 30 blob
Parsati almeno 40 blob
Parsati almeno 50 blob
Parsati almeno 60 blob
Parsati almeno 70 blob
Parsati almeno 80 blob
Parsati almeno 90 blob
Parsati almeno 100 blob
Parsati almeno 110 blob
Parsati almeno 120 blob
Parsati almeno 130 blob
Parsati almeno 140 blob
Parsati almeno 150 blob
Parsati almeno 160 blob
Parsati almeno 170 blob
Parsati almeno 180 blob
Parsati almeno 190 blob
Parsati almeno 200 blob
Parsati almeno 210 blob
Parsati almeno 220 blob
Parsati almeno 230 blob
Parsati almeno 240 blob
Parsati almeno 250 blob
Parsati almeno 260 blob
Parsati almeno 270 blob
Parsati almeno 280 blob
Parsati almeno 290 blob
Parsati almeno 300 blob
Parsati almeno 310 blob
Parsati almeno 320 blob
Parsati almeno 330 blob
Parsati almeno 340 blob
Parsati almeno 350 blob
Parsati almeno 360 blob
Parsati almeno 370 blob
Parsati almeno 380 blob
Parsati almeno 390 blob
Parsati almeno 400 blob
Parsati almeno 410 blob
Parsati almeno 420 b

Parsati almeno 3330 blob
Parsati almeno 3340 blob
Parsati almeno 3350 blob
Parsati almeno 3360 blob
Parsati almeno 3370 blob
Parsati almeno 3380 blob
Parsati almeno 3390 blob
Parsati almeno 3400 blob
Parsati almeno 3410 blob
Parsati almeno 3420 blob
Parsati almeno 3430 blob
Parsati almeno 3440 blob
Parsati almeno 3450 blob
Parsati almeno 3460 blob
Parsati almeno 3470 blob
Parsati almeno 3480 blob
Parsati almeno 3490 blob
Parsati almeno 3500 blob
Parsati almeno 3510 blob
Parsati almeno 3520 blob
Parsati almeno 3530 blob
Parsati almeno 3540 blob
Parsati almeno 3550 blob
Parsati almeno 3560 blob
Parsati almeno 3570 blob
Parsati almeno 3580 blob
Parsati almeno 3590 blob
Parsati almeno 3600 blob
Parsati almeno 3610 blob
Parsati almeno 3620 blob
Parsati almeno 3630 blob
Parsati almeno 3640 blob
Parsati almeno 3650 blob
Parsati almeno 3660 blob
Parsati almeno 3670 blob
Parsati almeno 3680 blob
Parsati almeno 3690 blob
Parsati almeno 3700 blob
Parsati almeno 3710 blob
Parsati almeno 3720 blob


Parsati almeno 6530 blob
Parsati almeno 6540 blob
Parsati almeno 6550 blob
Parsati almeno 6560 blob
Parsati almeno 6570 blob
Parsati almeno 6580 blob
Parsati almeno 6590 blob
Parsati almeno 6600 blob
Parsati almeno 6610 blob
Parsati almeno 6620 blob
Parsati almeno 6630 blob
Parsati almeno 6640 blob
Parsati almeno 6650 blob
Parsati almeno 6660 blob
Parsati almeno 6670 blob
Parsati almeno 6680 blob
Parsati almeno 6690 blob
Parsati almeno 6700 blob
Parsati almeno 6710 blob
Parsati almeno 6720 blob
Parsati almeno 6730 blob
Parsati almeno 6740 blob
Parsati almeno 6750 blob
Parsati almeno 6760 blob
Parsati almeno 6770 blob
Parsati almeno 6780 blob
Parsati almeno 6790 blob
Parsati almeno 6800 blob
Parsati almeno 6810 blob
Parsati almeno 6820 blob
Parsati almeno 6830 blob
Parsati almeno 6840 blob
Parsati almeno 6850 blob
Parsati almeno 6860 blob
Parsati almeno 6870 blob
Parsati almeno 6880 blob
Parsati almeno 6890 blob
Parsati almeno 6900 blob
Parsati almeno 6910 blob
Parsati almeno 6920 blob


Parsati almeno 9810 blob
Parsati almeno 9820 blob
Parsati almeno 9830 blob
Parsati almeno 9840 blob
Parsati almeno 9850 blob
Parsati almeno 9860 blob
Parsati almeno 9870 blob
Parsati almeno 9880 blob
Parsati almeno 9890 blob
Parsati almeno 9900 blob
Parsati almeno 9910 blob
Parsati almeno 9920 blob
Parsati almeno 9930 blob
Parsati almeno 9940 blob
Parsati almeno 9950 blob
Parsati almeno 9960 blob
Parsati almeno 9970 blob
Parsati almeno 9980 blob
Parsati almeno 9990 blob
Parsati almeno 10000 blob
Switch to page 2!176!MDAwMDg3IWluZm9tb2JpbGl0eS9UU1BfVk9VQ0hFUi9JTkZPTU9CSUxJVFkyNjY2NTYvMjAxOC85LzMwLzIwMThfMDlfMzBfMTFfNDBfNDZfNTUyMjYwNTcuanNvbiEwMDAwMjghOTk5OS0xMi0zMVQyMzo1OTo1OS45OTk5OTk5WiE-
5000
Parsati almeno 10010 blob
Parsati almeno 10020 blob
Parsati almeno 10030 blob
Parsati almeno 10040 blob
Parsati almeno 10050 blob
Parsati almeno 10060 blob
Parsati almeno 10070 blob
Parsati almeno 10080 blob
Parsati almeno 10090 blob
Parsati almeno 10100 blob
Parsati almeno 10110 blob
Parsati al

Parsati almeno 12900 blob
Parsati almeno 12910 blob
Parsati almeno 12920 blob
Parsati almeno 12930 blob
Parsati almeno 12940 blob
Parsati almeno 12950 blob
Parsati almeno 12960 blob
Parsati almeno 12970 blob
Parsati almeno 12980 blob
Parsati almeno 12990 blob
Parsati almeno 13000 blob
Parsati almeno 13010 blob
Parsati almeno 13020 blob
Parsati almeno 13030 blob
Parsati almeno 13040 blob
Parsati almeno 13050 blob
Parsati almeno 13060 blob
Parsati almeno 13070 blob
Parsati almeno 13080 blob
Parsati almeno 13090 blob
Parsati almeno 13100 blob
Parsati almeno 13110 blob
Parsati almeno 13120 blob
Parsati almeno 13130 blob
Parsati almeno 13140 blob
Parsati almeno 13150 blob
Parsati almeno 13160 blob
Parsati almeno 13170 blob
Parsati almeno 13180 blob
Parsati almeno 13190 blob
Parsati almeno 13200 blob
Parsati almeno 13210 blob
Parsati almeno 13220 blob
Parsati almeno 13230 blob
Parsati almeno 13240 blob
Parsati almeno 13250 blob
Parsati almeno 13260 blob
Parsati almeno 13270 blob
Parsati alme

...

Parsati almeno 19140 blob
Parsati almeno 19150 blob
Parsati almeno 19160 blob
Parsati almeno 19170 blob
Parsati almeno 19180 blob
Parsati almeno 19190 blob
Parsati almeno 19200 blob
Parsati almeno 19210 blob
Parsati almeno 19220 blob
Parsati almeno 19230 blob
Parsati almeno 19240 blob
Parsati almeno 19250 blob
Parsati almeno 19260 blob
Parsati almeno 19270 blob
Parsati almeno 19280 blob
Parsati almeno 19290 blob
Parsati almeno 19300 blob
Parsati almeno 19310 blob
Parsati almeno 19320 blob
Parsati almeno 19330 blob
Parsati almeno 19340 blob
Parsati almeno 19350 blob
Parsati almeno 19360 blob
Parsati almeno 19370 blob
Parsati almeno 19380 blob
Parsati almeno 19390 blob
Parsati almeno 19400 blob
Parsati almeno 19410 blob
Parsati almeno 19420 blob
Parsati almeno 19430 blob
Parsati almeno 19440 blob
Parsati almeno 19450 blob
Parsati almeno 19460 blob
Parsati almeno 19470 blob
Parsati almeno 19480 blob
Parsati almeno 19490 blob
Parsati almeno 19500 blob
Parsati almeno 19510 blob
Parsati alme

Parsati almeno 22220 blob
Parsati almeno 22230 blob
Parsati almeno 22240 blob
Parsati almeno 22250 blob
Parsati almeno 22260 blob
Parsati almeno 22270 blob
Parsati almeno 22280 blob
Parsati almeno 22290 blob
Parsati almeno 22300 blob
Parsati almeno 22310 blob
Parsati almeno 22320 blob
Parsati almeno 22330 blob
Parsati almeno 22340 blob
Parsati almeno 22350 blob
Parsati almeno 22360 blob
Parsati almeno 22370 blob
Parsati almeno 22380 blob
Parsati almeno 22390 blob
Parsati almeno 22400 blob
Parsati almeno 22410 blob
Parsati almeno 22420 blob
Parsati almeno 22430 blob
Parsati almeno 22440 blob
Parsati almeno 22450 blob
Parsati almeno 22460 blob
Parsati almeno 22470 blob
Parsati almeno 22480 blob
Parsati almeno 22490 blob
Parsati almeno 22500 blob
Parsati almeno 22510 blob
Parsati almeno 22520 blob
Parsati almeno 22530 blob
Parsati almeno 22540 blob
Parsati almeno 22550 blob
Parsati almeno 22560 blob
Parsati almeno 22570 blob
Parsati almeno 22580 blob
Parsati almeno 22590 blob
Parsati alme

Parsati almeno 23920 blob
Parsati almeno 23930 blob
Parsati almeno 23940 blob
Parsati almeno 23950 blob
Parsati almeno 23960 blob
Parsati almeno 23970 blob
Parsati almeno 23980 blob
Parsati almeno 23990 blob
Parsati almeno 24000 blob
Parsati almeno 24010 blob
Parsati almeno 24020 blob
Parsati almeno 24030 blob
Parsati almeno 24040 blob
Parsati almeno 24050 blob
Parsati almeno 24060 blob
Parsati almeno 24070 blob
Parsati almeno 24080 blob
Parsati almeno 24090 blob
Parsati almeno 24100 blob
Parsati almeno 24110 blob
Parsati almeno 24120 blob
Parsati almeno 24130 blob
Parsati almeno 24140 blob
Parsati almeno 24150 blob
Parsati almeno 24160 blob
Parsati almeno 24170 blob
Parsati almeno 24180 blob
Parsati almeno 24190 blob
Parsati almeno 24200 blob
Parsati almeno 24210 blob
Parsati almeno 24220 blob
Parsati almeno 24230 blob
Parsati almeno 24240 blob
Parsati almeno 24250 blob
Parsati almeno 24260 blob
Parsati almeno 24270 blob
Parsati almeno 24280 blob
Parsati almeno 24290 blob
Parsati alme

Parsati almeno 27000 blob
Parsati almeno 27010 blob
Parsati almeno 27020 blob
Parsati almeno 27030 blob
Parsati almeno 27040 blob
Parsati almeno 27050 blob
Parsati almeno 27060 blob
Parsati almeno 27070 blob
Parsati almeno 27080 blob
Parsati almeno 27090 blob
Parsati almeno 27100 blob
Parsati almeno 27110 blob
Parsati almeno 27120 blob
Parsati almeno 27130 blob
Parsati almeno 27140 blob
Parsati almeno 27150 blob
Parsati almeno 27160 blob
Parsati almeno 27170 blob
Parsati almeno 27180 blob
Parsati almeno 27190 blob
Parsati almeno 27200 blob
Parsati almeno 27210 blob
Parsati almeno 27220 blob
Parsati almeno 27230 blob
Parsati almeno 27240 blob
Parsati almeno 27250 blob
Parsati almeno 27260 blob
Parsati almeno 27270 blob
Parsati almeno 27280 blob
Parsati almeno 27290 blob
Parsati almeno 27300 blob
Parsati almeno 27310 blob
Parsati almeno 27320 blob
Parsati almeno 27330 blob
Parsati almeno 27340 blob
Parsati almeno 27350 blob
Parsati almeno 27360 blob
Parsati almeno 27370 blob
Parsati alme

Parsati almeno 30080 blob
Parsati almeno 30090 blob
Parsati almeno 30100 blob
Parsati almeno 30110 blob
Parsati almeno 30120 blob
Parsati almeno 30130 blob
Parsati almeno 30140 blob
Parsati almeno 30150 blob
Parsati almeno 30160 blob
Parsati almeno 30170 blob
Parsati almeno 30180 blob
Parsati almeno 30190 blob
Parsati almeno 30200 blob
Parsati almeno 30210 blob
Parsati almeno 30220 blob
Parsati almeno 30230 blob
Parsati almeno 30240 blob
Parsati almeno 30250 blob
Parsati almeno 30260 blob
Parsati almeno 30270 blob
Parsati almeno 30280 blob
Parsati almeno 30290 blob
Parsati almeno 30300 blob
Parsati almeno 30310 blob
Parsati almeno 30320 blob
Parsati almeno 30330 blob
Parsati almeno 30340 blob
Parsati almeno 30350 blob
Parsati almeno 30360 blob
Parsati almeno 30370 blob
Parsati almeno 30380 blob
Parsati almeno 30390 blob
Parsati almeno 30400 blob
Parsati almeno 30410 blob
Parsati almeno 30420 blob
Parsati almeno 30430 blob
Parsati almeno 30440 blob
Parsati almeno 30450 blob
Parsati alme

# prima esplorazione...

In [ ]:
missing_values = []
for root,_,files in os.walk('C:\\Users\leuci\Desktop\Blob2csv\POS'):
    for file in files:
        if 'before' in file:
            continue
        dff = pd.read_csv(os.path.join(root,file),delimiter=';')
        missing = len(dff) - len(dff.dropna())
        missing_values.append(missing)

In [ ]:
plt.hist(missing_values,bins=25)
plt.xlabel('numero di valori mancanti (per crash)')
plt.ylabel('frequenza')
plt.title('valori mancanti nei crash')

In [ ]:
L = 0
for root,_,files in os.walk('C:\\Users\leuci\Desktop\Blob2csv\POS'):
    for file in files:
        if 'before' in file:
            continue
        dff = pd.read_csv(os.path.join(root,file),delimiter=';')
        index = dff['latitude'].isnull()
        x = dff[index].index.values
        l = 1
        for i in range(len(x)-1):
            if x[i+1] == x[i]+1:
                l += 1
                if l>L:
                    L = l
            else:
                l = 0
        print(l, file)
L

In [ ]:
missing_values[missing_values!=0]

In [ ]:
data

In [ ]:
df_pos